In [28]:
import math
import numpy as np

In [29]:
EPS = 1e-7
ITERATIONS = 200000

In [30]:
def equalVector(a, b, eps):
    return math.sqrt(sum([(a[i] - b[i])**2 for i in range(len(a))])) < eps

In [31]:
def zeidelMethod(A, b):
    n = len(A)
    x = [.0 for i in range(n)]
    xn = [100 for i in range(n)]
    
    curIterations = 0
    while not equalVector(x, xn, EPS) and curIterations < ITERATIONS:
        x = np.copy(xn)
        curIterations += 1
        for i in range(n):
            s1 = - sum(A[i][j] * xn[j] for j in range(i)) / A[i][i]
            s2 = - sum(A[i][j] * x[j] for j in range(i + 1, n)) / A[i][i]
            xn[i] = (b[i] / A[i][i] + s1 + s2)
            
    print("Iterations = ", curIterations)
    return xn

In [32]:
def zeidelMethodRelax(A, b):
    n = len(A)
    x = [.0 for i in range(n)]
    xn = [100 for i in range(n)]
    Relax = 1.6
    curIterations = 0
    while not equalVector(x, xn, EPS) and curIterations < ITERATIONS:
        x = np.copy(xn)
        curIterations += 1
        for i in range(n):
            s1 = - sum(A[i][j] * xn[j] for j in range(i)) / A[i][i]
            s2 = - sum(A[i][j] * x[j] for j in range(i + 1, n)) / A[i][i]
            xn[i] = (b[i] / A[i][i] + s1 + s2)
            xn[i] = Relax * xn[i] + (1 - Relax) * x[i]
            
    print("Iterations = ", curIterations)
    return xn